In [3]:
#!pip install fitz
#!pip install pymupdf
import boto3
import fitz
from io import BytesIO
s3_client=boto3.client('s3')

In [4]:
def is_pdf_openable(bucket,key):
    try:
        response=s3_client.get_object(Bucket=bucket,Key=key)
        pdf_content=response['Body'].read()
        doc=fitz.open(stream=BytesIO(pdf_content))
        doc.close()
        return True
    except Exception as e:
        print(f"error opening {key}: {str(e)}")
        return False

In [27]:
page_dict=dict()

In [20]:
def page_count_dictionary(pagenum):
    if pagenum in page_dict:
        page_dict[pagenum] += 1
    else:
        page_dict[pagenum] = 1

In [5]:
s3_client = boto3.client('s3')
bucket_name = 's3sagemakerbucket'
count=0
pdf_files=[]
def read_pdf_files_from_folder(folder_path):
    global count
    response = s3_client.list_objects_v2(Bucket=bucket_name, Prefix=folder_path)
    for obj in response['Contents']:
        if obj['Key'].endswith('.pdf'):
            print(obj['Key'])
            if is_pdf_openable(bucket_name,obj['Key']):
                count=count+1
                pdf_files.append(obj['Key'])
            #print(obj['Key'].split('/')[-1])
            # s3_client.download_file(bucket_name, obj['Key'], 'local_directory\/' + obj['Key'])
        else:
            if obj['Key'].endswith('\/'):
                read_pdf_files_from_folder(obj['Key'])

root_folder = 'BondingLine_data/'
read_pdf_files_from_folder(root_folder)
print(count)

BondingLine_data/Identification 2/DACH/SE Austria_UniCredit Bank Austria_2006.pdf
BondingLine_data/Identification 2/DACH/SE Austria_UniCredit Bank Austria_2014.pdf
BondingLine_data/Identification 2/DACH/SE Austria_UniCredit Bank Austria_2016.pdf
BondingLine_data/Identification 2/DACH/SE Austria_UniCredit Bank Austria_2018.pdf
BondingLine_data/Identification 2/DACH/SE Gemany_Commerzbank_amendment nr._6_11.2016.pdf
BondingLine_data/Identification 2/DACH/SE Gemany_HSBC_2022.pdf
BondingLine_data/Identification 2/DACH/SE Gemany_HSBC_amendment1_2017.pdf
BondingLine_data/Identification 2/DACH/SE Gemany_HSBC_amendment2_2018.pdf
BondingLine_data/Identification 2/DACH/SE Gemany_HSBC_amendment3_2019.pdf
BondingLine_data/Identification 2/DACH/SE Gemany_HSBC_amendment4_2019.pdf
BondingLine_data/Identification 2/DACH/SE Gemany_Nordea_2016.pdf
BondingLine_data/Identification 2/DACH/SE Germany_Deutsche Bank_Areva.pdf
BondingLine_data/Identification 2/DACH/SE Switzerland_CH01_Deutsche Bank_amendment2_2

In [8]:
len(pdf_files)

101

In [9]:
!mkdir -p bondinglinefiles

In [10]:
for file in pdf_files:
    s3_client.download_file("s3sagemakerbucket",str(file),f"bondinglinefiles/{file.split('/')[-1]}",)

In [12]:
!pip install PyPDF2
from PyPDF2 import PdfReader, PdfWriter

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.6/232.6 kB 5.3 MB/s eta 0:00:00a 0:00:01

[notice] A new release of pip is available: 23.2.1 -> 23.3.2
[notice] To update, run: pip install --upgrade pip


In [28]:
import os
file_path="bondinglinefiles/"
extracted=0
for filename in os.listdir(file_path):
    inputpdf = PdfReader(open(f'bondinglinefiles/{filename}', "rb"))
    print(f"no of pages :{len(inputpdf.pages)}")
    page_count_dictionary(len(inputpdf.pages))
    extracted+=1
    
print(f"extracted files :{extracted}")

no of pages :6
no of pages :14
no of pages :4
no of pages :56
no of pages :22
no of pages :2
no of pages :10
no of pages :12
no of pages :3
no of pages :26
no of pages :2
no of pages :12
no of pages :10
no of pages :12
no of pages :10
no of pages :7
no of pages :2
no of pages :18
no of pages :8
no of pages :1
no of pages :32
no of pages :4
no of pages :4
no of pages :7
no of pages :3
no of pages :20
no of pages :26
no of pages :4
no of pages :2
no of pages :71
no of pages :3
no of pages :15
no of pages :1
no of pages :5
no of pages :6
no of pages :3
no of pages :10
no of pages :5
no of pages :2
no of pages :20
no of pages :8
no of pages :4
no of pages :4
no of pages :5
no of pages :3
no of pages :8
no of pages :5
no of pages :10
no of pages :8
no of pages :12
no of pages :14
no of pages :7
no of pages :11
no of pages :3
no of pages :11
no of pages :9
no of pages :4
no of pages :4
no of pages :9
no of pages :2
no of pages :4
no of pages :18
no of pages :17
no of pages :8
no of pages :26

In [29]:
print(len(page_dict))

26


In [30]:
print(page_dict)

{6: 6, 14: 2, 4: 11, 56: 2, 22: 1, 2: 13, 10: 8, 12: 6, 3: 10, 26: 3, 7: 3, 18: 2, 8: 7, 1: 3, 32: 1, 20: 3, 71: 1, 15: 2, 5: 5, 11: 3, 9: 3, 17: 1, 21: 1, 16: 2, 13: 1, 19: 1}


In [32]:
total=0
for ind,val in page_dict.items():
    print(f"pdf with {ind} pages : {val}") 
    total+=val
print(total)

pdf with 6 pages : 6
pdf with 14 pages : 2
pdf with 4 pages : 11
pdf with 56 pages : 2
pdf with 22 pages : 1
pdf with 2 pages : 13
pdf with 10 pages : 8
pdf with 12 pages : 6
pdf with 3 pages : 10
pdf with 26 pages : 3
pdf with 7 pages : 3
pdf with 18 pages : 2
pdf with 8 pages : 7
pdf with 1 pages : 3
pdf with 32 pages : 1
pdf with 20 pages : 3
pdf with 71 pages : 1
pdf with 15 pages : 2
pdf with 5 pages : 5
pdf with 11 pages : 3
pdf with 9 pages : 3
pdf with 17 pages : 1
pdf with 21 pages : 1
pdf with 16 pages : 2
pdf with 13 pages : 1
pdf with 19 pages : 1
101
